In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [1]:
!pip install python-dotenv

In [15]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Milvus
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from IPython.display import display, Markdown

In [10]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [21]:
%pip install pymilvus==2.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 420.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 948.4 kB/s eta 0:00:0000:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 804.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 332.7 kB/s eta 0:00:00 0:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.60.0
    Uninstalling grpcio-1.60.0:
      Successfully uninstalled grpcio-1.60.0
  Attempting uninstall: pymilvus
    Found existing installation: pymilvus 2.3.6
    Uninstalling pymilvus-2.3.6:
      Successfully uninstalled pymilvus-2.3.6
Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain_community.document_loaders import TextLoader

# loader = TextLoader("../data/Robinson_Advisory.txt", encoding="windows-1252")
loader = TextLoader("../data/Robinson_Advisory.docx", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

RuntimeError: Error loading ../data/Robinson_Advisory.docx

In [4]:
vector_db = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"host": "192.168.137.236", "port": "19530"},
)

In [5]:
query = "Who are the parties to the Agreement and what are their defined names?"
docs = vector_db.similarity_search(query)

In [6]:
docs[0].page_content

'9. Entire Agreement; No Waiver or Assignment: This Agreement together with the Exhibits, which are attached hereto and incorporated herein, set forth the entire Agreement between the parties and shall supersede all previous communications and agreements between the parties, either oral or written. This Agreement may be modified only by a written amendment executed by both parties. This Agreement may not be assigned, sold, delegated or transferred in any manner by Advisor for any reason whatsoever. The Company may assign the Agreement to a successor of all or substantially all of its assets or business, provided the assignee has assumed the Company’s obligations under this Agreement.'

In [11]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [13]:
response = llm.call_as_llm(f"{qdocs} Question: Who are the parties to the Agreement and what are their defined names?") 


/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `call_as_llm` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [16]:
display(Markdown(response))

The parties to the Agreement are Cloud Investments Ltd. (referred to as "Company" in the Agreement) and an Advisor (referred to as "Advisor" in the Agreement), whose name is Jack Robinson.

In [20]:
query = "What is the termination notice?"
docs = vector_db.similarity_search(query)

In [21]:
docs[0].page_content

'4. Term: The term of this Agreement shall commence on the Effective Date and shall continue until terminated in accordance with the provisions herein (the "Term").  \n\n5. Termination: Either party, at any given time, may terminate this Agreement, for any reason whatsoever, with or without cause, upon fourteen (14) days’ prior written notice. Notwithstanding the above, the Company may terminate this Agreement immediately and without prior notice if Advisor refuses or is unable to perform the Services, or is in breach of any provision of this Agreement.'

In [22]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])
response = llm.call_as_llm(f"{qdocs} Question: What is the termination notice?") 

In [23]:
display(Markdown(response))

The termination notice is fourteen (14) days' prior written notice, which can be given by either party at any time, for any reason, with or without cause. However, the Company may terminate the Agreement immediately and without prior notice if the Advisor refuses or is unable to perform the Services or is in breach of any provision of the Agreement.